In [5]:
class Node:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        
class BinarySearchTree:
    def __init__(self):
        self.root = None
        
    def insert(self, key):
        """데이터를 삽입하는 함수"""
        self.root = self._insert_recursive(self.root, key)
        
    def _insert_recursive(self, node, key):
        if node is None:
            return Node(key)
        
        if key < node.key:
            node.left = self._insert_recursive(node.left, key)
        else:
            node.right = self._insert_recursive(node.right, key)
        return node
        
    def search(self, key):
        return self._search_recursive(self.root, key)
    
    def _search_recursive(self, node, key):
        if node is None or node.key == key:
            return node
        
        if key < node.key:
            return self._search_recursive(node.left, key)
        else:
            return self._search_recursive(node.right, key)
        
    def delete(self, key):
        self.root = self._delete_recursive(self.root, key)
        
    def _delete_recursive(self, node, key):
        if node is None:
            return node
        if key < node.key:
            node.left = self._delete_recursive(node.left, key)
        elif key > node.key:
            node.right = self._delete_recursive(node.right, key)
        else:
            if node.left is None:
                return node.right
            elif node.right is None:
                return node.left
            
            temp = self._min_value_node(node.right)
            node.key = temp.key
            node.right = self._delete_recursive(node.right, temp.key)
            
        return node
    
    def _min_value_node(self, node):
        """오른쪽 서브트리에서 가장 작은 값을 찾는 헬퍼 함수"""
        current = node
        while current.left is not None:
            current = current.left
        return current
    
    def inorder(self):
        """중위 순회 - BST는 중위 순회하면 정렬된 결과가 나옴"""
        self._inorder_recursive(self.root)
        print()
        
    def _inorder_recursive(self, node):
        if node:
            self._inorder_recursive(node.left)
            print(node.key, end=' ')
            self._inorder_recursive(node.right)
        
def practice_bst():
    print("=== 이진 탐색 트리(BST) 실습 ===")
    bst = BinarySearchTree()
    
    # 데이터 삽입: 50 -> 30 -> 20 -> 40 -> 70 -> 60 -> 80
    numbers = [50, 30, 20, 40, 70, 60, 80]
    for num in numbers:
        bst.insert(num)
    
    print("초기 트리 (중위 순회): ", end="")
    bst.inorder() # 20 30 40 50 60 70 80 (정렬되어 나와야 함)
    
    # 탐색
    print(f"Search 40: {'Found' if bst.search(40) else 'Not Found'}")
    print(f"Search 99: {'Found' if bst.search(99) else 'Not Found'}")

    # 삭제 Case 1: 리프 노드 삭제 (20)
    print("\nDelete 20 (Leaf Node):")
    bst.delete(20)
    bst.inorder()

    # 삭제 Case 2: 자식이 하나인 노드 삭제 (30은 40을 가지고 있음)
    # 현재 구조상 30의 오른쪽 자식은 40
    print("Delete 30 (One Child):")
    bst.delete(30)
    bst.inorder()

    # 삭제 Case 3: 자식이 둘인 노드 삭제 (50은 40과 70을 가짐)
    # 50이 삭제되면 오른쪽(70,60,80) 중 가장 작은 60이 올라와야 함
    print("Delete 50 (Root, Two Children):")
    bst.delete(50)
    bst.inorder()

practice_bst()

=== 이진 탐색 트리(BST) 실습 ===
초기 트리 (중위 순회): 20 30 40 50 60 70 80 
Search 40: Found
Search 99: Not Found

Delete 20 (Leaf Node):
30 40 50 60 70 80 
Delete 30 (One Child):
40 50 60 70 80 
Delete 50 (Root, Two Children):
40 60 70 80 


In [6]:
def check_skewed_tree():
    print("\n=== 편향 트리(Skewed Tree) 문제 확인 ===")
    bst = BinarySearchTree()
    
    # 정렬된 데이터를 순서대로 넣으면?
    skewed_data = [10, 20, 30, 40, 50]
    for num in skewed_data:
        bst.insert(num)
        
    print("데이터가 10->20->30->40->50 순으로 들어감")
    print("Root:", bst.root.key)
    print("Root.right:", bst.root.right.key)
    print("Root.right.right:", bst.root.right.right.key)
    print("--> 트리가 오른쪽으로만 길게 뻗어버림 (사실상 연결 리스트)")
    print("--> 이 경우 검색 성능이 O(log n)이 아니라 O(n)으로 떨어짐")
    print("--> 해결책: 강의 자료 63페이지의 'AVL Tree' (회전 연산으로 균형 맞춤)")

check_skewed_tree()


=== 편향 트리(Skewed Tree) 문제 확인 ===
데이터가 10->20->30->40->50 순으로 들어감
Root: 10
Root.right: 20
Root.right.right: 30
--> 트리가 오른쪽으로만 길게 뻗어버림 (사실상 연결 리스트)
--> 이 경우 검색 성능이 O(log n)이 아니라 O(n)으로 떨어짐
--> 해결책: 강의 자료 63페이지의 'AVL Tree' (회전 연산으로 균형 맞춤)


In [7]:
class Node:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.height = 1  # 높이 정보를 노드에 저장해야 함

class AVLTree:
    def get_height(self, node):
        """노드의 높이 반환 (None이면 0)"""
        if not node:
            return 0
        return node.height

    def get_balance(self, node):
        """균형 인수(Balance Factor) 계산"""
        if not node:
            return 0
        return self.get_height(node.left) - self.get_height(node.right)

    def right_rotate(self, y):
        """우회전 (Right Rotation) - Slide 63"""
        print(f"🔄 Right Rotation at {y.key}")
        x = y.left
        T2 = x.right

        # 회전 수행 (포인터 변경)
        x.right = y
        y.left = T2

        # 높이 갱신 (부모가 바뀌었으므로)
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        x.height = 1 + max(self.get_height(x.left), self.get_height(x.right))

        return x # 새로운 루트 반환

    def left_rotate(self, x):
        """좌회전 (Left Rotation)"""
        print(f"🔄 Left Rotation at {x.key}")
        y = x.right
        T2 = y.left

        # 회전 수행
        y.left = x
        x.right = T2

        # 높이 갱신
        x.height = 1 + max(self.get_height(x.left), self.get_height(x.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))

        return y

    def insert(self, root, key):
        """데이터 삽입 (재귀)"""
        
        # 1. 일반적인 BST 삽입 수행
        if not root:
            return Node(key)
        elif key < root.key:
            root.left = self.insert(root.left, key)
        else:
            root.right = self.insert(root.right, key)

        # 2. 노드 높이 갱신
        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))

        # 3. 균형 인수 계산 (비정상인지 확인)
        balance = self.get_balance(root)

        # 4. 불균형 발생 시 4가지 경우에 따라 회전 (Slide 63~66)

        # Case 1: Left Left (LL) - 오른쪽으로 회전
        if balance > 1 and key < root.left.key:
            return self.right_rotate(root)

        # Case 2: Right Right (RR) - 왼쪽으로 회전
        if balance < -1 and key > root.right.key:
            return self.left_rotate(root)

        # Case 3: Left Right (LR) - 왼쪽 회전 후 오른쪽 회전
        if balance > 1 and key > root.left.key:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        # Case 4: Right Left (RL) - 오른쪽 회전 후 왼쪽 회전
        if balance < -1 and key < root.right.key:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root

    def preorder(self, root):
        if root:
            print(f"{root.key}", end=" ")
            self.preorder(root.left)
            self.preorder(root.right)

# === 실행 테스트 ===
def test_avl_tree():
    print("=== AVL 트리 균형 유지 테스트 ===")
    avl = AVLTree()
    root = None

    # 편향된 데이터 입력: 10 -> 20 -> 30 -> 40 -> 50
    # 일반 BST였다면 일자 막대기(Skewed)가 되었을 것임
    data = [10, 20, 30, 40, 50]
    
    for key in data:
        print(f"\n[Insert {key}]")
        root = avl.insert(root, key)
        print("Current Preorder:", end=" ")
        avl.preorder(root)
        print()

    print("\n\n=== 최종 트리 구조 확인 ===")
    print(f"Root Node: {root.key}") # AVL이라면 20이 Root가 되어야 함 (중간값)
    print(f"Left Child: {root.left.key}")
    print(f"Right Child: {root.right.key}")

test_avl_tree()

=== AVL 트리 균형 유지 테스트 ===

[Insert 10]
Current Preorder: 10 

[Insert 20]
Current Preorder: 10 20 

[Insert 30]
🔄 Left Rotation at 10
Current Preorder: 20 10 30 

[Insert 40]
Current Preorder: 20 10 30 40 

[Insert 50]
🔄 Left Rotation at 30
Current Preorder: 20 10 40 30 50 


=== 최종 트리 구조 확인 ===
Root Node: 20
Left Child: 10
Right Child: 40
